In [1]:
using Pkg

using StaticArrays,BenchmarkTools,Random,Flux,DelimitedFiles,LinearAlgebra

# Symmetry Functions

Time to write and test the basics of SymmFunctions for ML potentials

In [5]:
distance2(a,b) = (a-b)⋅(a-b)

get_distance2_mat(pos) = [distance2(a,b) for a in pos, b in pos]

abstract type AbstractSymmFunction{T} end 
abstract type RadialSymmFunction{T} <: AbstractSymmFunction{T} end
abstract type AngularSymmFunction{T} <: AbstractSymmFunction{T} end



In [14]:
struct RadialType2{T} <: RadialSymmFunction{T}
    eta::T
    r_cut::T
end

function cutoff_function(r_scaled)
    
    cutoff= 0.5*(cos(π*r_scaled) + 1)
    

    return cutoff
end


cutoff_function (generic function with 1 method)

In [7]:
atoms = [[0.0000006584,       -0.0000019175,        0.0000000505],
[-0.0000005810,       -0.0000004871,        0.6678432175],
[0.1845874248,       -0.5681026047,        0.2986701538],
[-0.4832557457,       -0.3511072166,        0.2986684497],
[-0.4832557570,        0.3511046452,        0.2986669456],
[0.1845874064,        0.5681000550,        0.2986677202],
[0.5973371920,       -0.0000012681,        0.2986697030],
[-0.1845860897,       -0.5681038901,       -0.2986676192],
[-0.5973358752,       -0.0000025669,       -0.2986696020],
[-0.1845861081,        0.5680987696,       -0.2986700528],
[0.4832570624,        0.3511033815,       -0.2986683486],
[0.4832570738,       -0.3511084803,       -0.2986668445],
[0.0000018978,       -0.0000033480,       -0.6678431165],
[-0.0000017969,        0.0000009162,        1.3230014650],
[0.1871182835,       -0.5758942175,        0.9797717078],
[-0.4898861924,       -0.3559221410,       0.9797699802],
[-0.4898862039,        0.3559224872,        0.9797684555],
[0.1871182648,        0.5758945856,        0.9797692407],
[0.6055300485,        0.0000001908,        0.9797712507],
[0.7926501864,       -0.5758950093,        0.6055339635],
[0.3656681761,       -1.1254128670,        0.5916673591],
[-0.3027660545,       -0.9318173412,        0.6055326929],
[-0.9573332453,       -0.6955436707,        0.5916639831],
[-0.9797705418,       -0.0000006364,        0.6055294407],
[-0.9573332679,        0.6955423392,        0.5916610035],
[-0.3027660847,        0.9318160902,        0.6055287012],
[0.3656681396,        1.1254115783,        0.5916625380],
[0.7926501677,        0.5758937939,        0.6055314964],
[1.1833279992,       -0.0000006311,        0.5916664660],
[0.6770051458,       -0.9318186223,        0.0000033028],
[0.0000006771,       -1.1517907207,        0.0000025175],
[-0.6770037988,       -0.9318186442,        0.0000007900],
[-1.0954155825,       -0.3559242494,       -0.0000012200],
[-1.0954155940,        0.3559203788,       -0.0000027447],
[-0.6770038290,        0.9318147872,       -0.0000032017],
[0.0000006397,        1.1517868856,       -0.0000024165],
[0.6770051155,        0.9318148091,       -0.0000006889],
[1.0954168993,        0.3559204143,        0.0000013211],
[1.0954169108,       -0.3559242139,        0.0000028458],
[0.3027674014,       -0.9318199253,       -0.6055286002],
[-0.3656668229,       -1.1254154134,       -0.5916624370],
[-0.7926488510,       -0.5758976290,       -0.6055313954],
[-1.1833266824,       -0.0000032040,       -0.5916663649],
[-0.7926488697,        0.5758911742,       -0.6055338624],
[-0.3656668594,        1.1254090319,       -0.5916672580],
[0.3027673712,        0.9318135061,       -0.6055325919],
[0.9573345621,        0.6955398357,       -0.5916638820],
[0.9797718586,       -0.0000031986,       -0.6055293396],
[0.9573345846,       -0.6955461743,       -0.5916609025],
[-0.1871169480,       -0.5758984207,       -0.9797691397],
[-0.6055287318,       -0.0000040259,       -0.9797711497],
[-0.1871169667,        0.5758903824,       -0.9797716067],
[0.4898875091,        0.3559183059,       -0.9797698792],
[0.4898875207,       -0.3559263223,       -0.9797683545],
[0.0000031136,       -0.0000047513,       -1.3230013639]]*18.8973*0.36258

positions = [SVector{3}(p[i] for i in 1:3) for p in atoms]
dis2mat = get_distance2_mat(positions)
test_symm_func = RadialType2{Float64}(0.001,11.338)

RadialType2{Float64}(0.001, 11.338)

In [8]:
[sum(col) for col in eachcol(dis2mat)]

55-element Vector{Float64}:
 3105.759662745963
 4257.405302945296
 4257.405302938711
 4257.405303178946
 4257.405302780715
 4257.405302903702
 4257.405303032689
 4257.405303060748
 4257.405302948559
 4257.405303021082
    ⋮
 7625.260915818566
 6531.193620513525
 7625.260915728348
 6531.193620457171
 6531.193620670761
 6531.193620385166
 6531.193620679188
 6531.193620943284
 7625.260915313037

In [15]:
function calc_one_symm_function(r2_ij,eta,r_cut)
    if r2_ij != 0 && r2_ij < r_cut^2 
        g_ij = exp(-eta*(r2_ij)) * cutoff_function( sqrt(r2_ij)/r_cut )

        return g_ij
    else
        return 0.
    end
end

function calc_symm_functions(dist2_matrix,index,symmfunc::RadialType2)
    eta = symmfunc.eta
    r_cut = symmfunc.r_cut
    
    gvec = calc_one_symm_function.(dist2_matrix[index,:],eta,r_cut)
    return sum(gvec)   
end

calc_symm_functions (generic function with 1 method)

In [16]:
g_vec = calc_one_symm_function.(dis2mat[1,:],test_symm_func.eta,test_symm_func.r_cut)
println(g_vec)

[0.0, 0.6356829918738802, 0.6356829918598722, 0.6356829917848361, 0.6356829919344302, 0.6356829919381135, 0.635682991883609, 0.6356829918336206, 0.635682991883609, 0.6356829919131138, 0.6356829919421304, 0.6356829919271504, 0.63568299187388, 0.08837002965586485, 0.19840099523397722, 0.198400995248088, 0.1984009952727969, 0.19840099533090572, 0.19840099531534228, 0.19840099525221916, 0.08837002964825506, 0.19840099522530136, 0.08837002966090446, 0.19840099527685062, 0.08837002965683131, 0.1984009953155797, 0.08837002972195088, 0.1984009953298159, 0.08837002966898236, 0.1984009952196931, 0.19840099526253407, 0.1984009952481719, 0.19840099526537777, 0.19840099531122038, 0.19840099528032984, 0.1984009953374849, 0.1984009953528635, 0.1984009952885388, 0.19840099528805946, 0.19840099527464533, 0.08837002966266858, 0.19840099524075996, 0.08837002969250582, 0.1984009952775181, 0.0883700297019845, 0.19840099530563973, 0.08837002968442777, 0.1984009953162545, 0.0883700296672394, 0.19840099529343

In [17]:
sum(g_vec)

14.640666117305633

In [18]:
[calc_symm_functions(dis2mat,i,test_symm_func) for i in 1:55]

55-element Vector{Float64}:
 14.640666117305633
 11.966607067855739
 11.966607067541357
 11.966607067163583
 11.966607068360855
 11.966607068035858
 11.96660706756069
 11.966607067316641
 11.966607067815215
 11.96660706776792
  ⋮
  5.6995369746290425
  7.455371452799304
  5.699536974721713
  7.455371453127722
  7.455371452781338
  7.455371453231261
  7.455371452614264
  7.455371452215095
  5.699536975557825

In [19]:
function calc_symm_functions(dist2_matrix,index,symmfunc::RadialType2,g_min,g_max)
    eta = symmfunc.eta
    r_cut = symmfunc.r_cut
    gvec = calc_one_symm_function.(dist2_matrix[index,:],eta,r_cut)
    g_unscaled = sum(gvec)

    return (g_unscaled - g_min)/(g_max - g_min)   
end

calc_symm_functions (generic function with 2 methods)

In [20]:
[calc_symm_functions(dis2mat,i,test_symm_func,0.450564942,15.523345849) for i in 1:55]

55-element Vector{Float64}:
 0.9414388269065572
 0.7640290266879376
 0.76402902666708
 0.7640290266420167
 0.7640290267214493
 0.7640290266998875
 0.7640290266683626
 0.7640290266521712
 0.764029026685249
 0.7640290266821113
 ⋮
 0.34824177867478656
 0.4647321920234493
 0.34824177868093475
 0.46473219204523813
 0.46473219202225735
 0.46473219205210736
 0.46473219201117283
 0.46473219198469007
 0.3482417787364064